# Python speed studies
This notebook uses the %timeit magic to measure and compare the performance of some large operations.

Credit: Nick Smith (Fermilab)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numba
import pandas as pd
import time

In [ ]:
%%timeit
# Use %%timeit to time a whole cell
x=1

In [ ]:
# Use %timeit to time a single command
%timeit y=[1, 2]

## Some simple array operations

In [ ]:
%timeit np.empty(100_000)
%timeit np.zeros(100_000)
%timeit np.arange(100_000)

In [ ]:
%%timeit
a = np.arange(100_000)
b = a*2

In [ ]:
%%timeit
c = list(range(100_000))
d = [i*2 for i in c]

## Aside: saving the results of timeit
The next two cells show how you can capture the results from the %timeit magic and make a plot.

In [ ]:
xx = np.array([10**i for i in range(8)])
yy = np.zeros_like(xx, dtype=float)
yy_err = np.zeros_like(xx, dtype=float)
for i in range(len(xx)):
    result = %timeit -o np.arange(xx[i])
    yy[i] = result.average
    yy_err[i] = result.stdev
    #yy[imax] = timeitresult.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.errorbar(xx, yy*1.e9, yerr=yy_err*1.e9)
ax.set_xlabel("Array length")
ax.set_xscale("log")
ax.set_ylabel("Time [ns]")
ax.set_yscale("log")


## numba
Some basic numba example from the numba tutorial

In [ ]:
from numba import jit

x = np.arange(100).reshape(10, 10)

def go_slow(a): # Function is compiled to machine code when called the first time
    trace = 0.0
    for i in range(a.shape[0]):   
        trace += np.tanh(a[i, i]) 
    return a + trace              

start = time.time()
go_slow(x)
end = time.time()
print(f"go_slow() = {(end - start)*1e6:.4e} us")


@numba.jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = 0.0
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
go_fast(x)
end = time.time()
print(f"Elapsed (with compilation) = {(end - start)*1e3:.4e} ms")

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
go_fast(x)
end = time.time()
print(f"Elapsed (after compilation) = {(end - start)*1e6:.4e} us")

start = time.time()
go_fast(x)
end = time.time()
print(f"Elapsed (third try) = {(end - start)*1e6:.4e} us")



In [ ]:
# A big example
x = np.arange(1_000_000).reshape(1000, 1000)
print("go_slow():")
%timeit go_slow(x)

print("go_fast():")
%timeit go_fast(x)

## Barnsley fern
The Barnsley fern is a fractal that you can generate with some simple matrix multiplications. See https://en.wikipedia.org/wiki/Barnsley_fern. 

Starting from an initial point, say $(x_0,\,y_0)=(0,\,0)$, the next point is generated by randomly applying one of four functions, $f_i(x, y)$:

$$
(x_{n+1}, y_{n+1}) = f_i(x_n, y_n)
$$

where each $f_i(x, y)$ consists of a matrix multiplication and addition:

$$
f_i(x, y) = \left[\begin{array}{cc} a & b \\ c & d \end{array} \right] \left[\begin{array}{c} x \\ y \end{array}\right] + \left[\begin{array}{c} e \\ f \end{array} \right].
$$

The coefficients and probabilities $p$ are:
| $w$ | $a$ | $b$ | $c$ | $d$ | $e$ | $f$ | $p$ | Portion generated | 
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| $f_1$ | $0$ | $0$ | $0$ | $0.16$ | $0$ | $0$ | $0.01$ | Stem |
| $f_2$ | $0.85$ | $0.04$ | $-0.04$ | $0.85$ | $0$ | $1.60$ | $0.85$ | Successively smaller leaflets |
| $f_3$ | $0.20$ | $-0.26$ | $0.23$ | $0.22$ | $0$ | $1.60$ | $0.07$ | Largest left-hand leaflet |
| $f_4$ | $-0.15$ | $0.28$ | $0.26$ | $0.24$ | $0$ | $0.44$ | $0.07$ | Largest right-hand leaflet |

The first cell below creates the arrays corresponding to the matrices (abcd) and offset (ef), plus an array for the probabilities. The next three cells run through 10,000 iterations using three different methods:

1. A simple for loop;
2. A vectorized for loop with 100 operations done in parallel;
3. A for loop compiled with numba.

Finally, we run 1,000,000 iterations and draw the fern.

In [ ]:
matrices = np.array([
    [[0,	0],	[0,	0.16]],
    [[0.85,	0.04],	[-0.04,	0.85]],
    [[0.20,	-0.26],	[0.23,	0.22]],
    [[-0.15, 0.28],	[0.26,	0.24]],
])
offsets = np.array([
    [0, 0],
    [0, 1.6],
    [0, 1.6],
    [0, 0.44],
])
probabilities = np.array([0.01, 0.85, 0.07, 0.07])

In [ ]:
%%timeit

# 1. Do the iteration in a for loop. Each iteration does a matrix multiplication to compute the next point in the series.
# David's result: 55.2 ms ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

current = np.array([0, 0])
allpoints = np.empty(shape=(10_000, 2))

for i in range(10_000):
    index = np.random.choice(4, p=probabilities)
    current = matrices[index] @ current + offsets[index]
    allpoints[i] = current

x = allpoints[:, 0]
y = allpoints[:, 1]

In [ ]:
%%timeit
# 2. Similar to the previous cell, but doing 100 operations at a time (i.e. in parallel)
# David's result: 1.11 ms ± 18.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

nparallel = 100
ntotal = 10_000
nsteps = ntotal // nparallel
current = np.zeros((nparallel, 2))
allpoints = np.empty(shape=(nsteps, nparallel, 2))

for i in range(nsteps):
    index = np.random.choice(4, size=nparallel, p=probabilities)
    current = np.einsum("ijk,ik->ij", matrices[index], current) + offsets[index]
    allpoints[i] = current

x = allpoints.reshape((-1, 2))[:, 0]
y = allpoints.reshape((-1, 2))[:, 1]

In [ ]:
# 3. Use numba to do "just in time" compilation (JIT) of the matrix multiplication function
cumprob = np.zeros(5)
cumprob[1:] = np.cumsum(probabilities)

# np.searchsorted(arr, x) returns the index where you would insert x into arr, such that the order of arr is maintained
# In this example, this converts a random number in the interval [0, 1] to an index (0, 1, 2, or 3) according to the specified probabilities
@numba.jit
def numba_choice(n, rng):
    return np.searchsorted(cumprob, rng.random(n)) - 1


@numba.jit
def build_fern(allpoints, rng):
    nsteps = len(allpoints)
    
    for i in range(1, nsteps):
        index = numba_choice(1, rng)[0]
        allpoints[i] = matrices[index] @ allpoints[i-1] + offsets[index]
    
    x = allpoints[:, 0]
    y = allpoints[:, 1]
    return x, y




In [ ]:
%%timeit 
# David's result: 1.27 ms ± 23 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
allpoints = np.empty(shape=(10_000, 2))
rng = np.random.default_rng(42)
x, y = build_fern(allpoints, rng)


## Drawing the fern


In [ ]:
# Now do it for real
allpoints = np.zeros(shape=(100, 2))
rng = np.random.default_rng(42)
x, y = build_fern(allpoints, rng)
%timeit x, y = build_fern(allpoints, rng)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x, y, marker=",", s=(72/fig.dpi)**2, lw=0, color="green")

Try modifying the matrix/offset numbers and see what happens!

In [ ]:
matrices2 = np.array([
    [[0,	0],	[0,	0.25]],
    [[0.95,	0.005],	[-0.005, 0.93]],
    [[0.035, -0.2],	[0.16,	0.04]],
    [[-0.04, 0.2],	[0.16,	0.04]],
])
offsets2 = np.array([
    [0, -0.4],
    [-0.002, 0.5],
    [-0.09, 0.02],
    [0.083, 0.12],
])
probabilities2 = np.array([0.02, 0.84, 0.07, 0.07])

cumprob2 = np.zeros(5)
cumprob2[1:] = np.cumsum(probabilities2)

# np.searchsorted(arr, x) returns the index where you would insert x into arr, such that the order of arr is maintained
# In this example, this converts a random number in the interval [0, 1] to an index (0, 1, 2, or 3) according to the specified probabilities
@numba.jit
def numba_choice(n, rng):
    return np.searchsorted(cumprob2, rng.random(n)) - 1


@numba.jit
def build_fern(allpoints, rng):
    nsteps = len(allpoints)
    
    for i in range(1, nsteps):
        index = numba_choice(1, rng)[0]
        allpoints[i] = matrices2[index] @ allpoints[i-1] + offsets2[index]
    
    x = allpoints[:, 0]
    y = allpoints[:, 1]
    return x, y


allpoints2 = np.zeros(shape=(100000, 2))
rng2 = np.random.default_rng(42)
x2, y2 = build_fern(allpoints2, rng2)


In [ ]:
fig, ax = plt.subplots()
ax.scatter(x2, y2, marker=",", s=(72/fig.dpi)**2, lw=0, color="green")